# Evaluations

 - use groq open source models for local app implementations(the system itself only use
 - use Claude 3 to 1. tidy ground truth data, and 2. evaluate as judge

In [ ]:
import os
os.environ["GROQ_API_KEY"] = "YOUR API KEY"

os.environ["ANTHROPIC_API_KEY"] = "YOUR API KEY"


In [2]:
from claude import claude
from groq_local import llm as local_llm

In [3]:
first_run = False
import pandas as pd

if first_run:
    from datasets import load_dataset


    dataset = load_dataset("har1/MTS_Dialogue-Clinical_Note")
    df = pd.DataFrame(dataset['train'])
    df.to_csv("MTS_clinical_note.csv", index = False)



    import numpy as np
    np.random.seed(3)   

    from sklearn.model_selection import train_test_split

    train_indices, test_indices = train_test_split(df.index, test_size=64, random_state=3)

    train_df = df.loc[train_indices]
    test_df = df.loc[test_indices]

    train_df.to_csv("train_" + "MTS_clinical_note.csv", index = False)
    test_df.to_csv("test_" + "MTS_clinical_note.csv", index = False)

else:
    train_df = pd.read_csv("train_" + "MTS_clinical_note.csv", )
    test_df = pd.read_csv("test_" + "MTS_clinical_note.csv", )

train_df = train_df.reset_index()

/home/hao/anaconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 0 examples [00:00, ? examples/s]/home/hao/anaconda3/envs/llm/lib/python3.10/site-packages/datasets/download/streaming_download_manager.py:778: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)
/home/hao/anaconda3/envs/llm/lib/python3.10/site-packages/datasets/download/streaming_download_manager.py:778: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)
Generating train split: 1301 examples [00:00, 11249

In [4]:
train_df

,index,ID,section_header,section_text,dialogue
0,137,137,PASTMEDICALHX,Symptoms: N/A.\r\nDiagnosis: N/A.\r\nHistory o...,Doctor: Have you ever been diagnosed with any ...
1,148,148,GENHX,Symptoms: N/A.\r\nDiagnosis: N/A.\r\nHistory o...,Doctor: How long it has been since these psych...
2,803,803,FAM/SOCHX,Symptoms: N/A\r\nDiagnosis: N/A\r\nHistory of ...,"Doctor: Do you drink?\nPatient: Like, every da..."
3,355,355,GENHX,"Symptoms: Fever, congestion, decreased urine o...",Doctor: How's the little dude?\nGuest_family: ...
4,228,228,PASTMEDICALHX,"Symptoms: N/A\nDiagnosis: Atrial fibrillation,...","Doctor: So, I see here in your reports you hav..."
...,...,...,...,...,...
1232,789,789,CC,Symptoms: Severe bruising\r\nDiagnosis: Probab...,Doctor: What brings you back into the clinic t...
1233,256,256,FAM/SOCHX,Symptoms: N/A\r\nDiagnosis: N/A\r\nHistory of ...,Doctor: Tell me about your family medical hist...
1234,968,968,FAM/SOCHX,Symptoms: N/A\r\nDiagnosis: N/A\r\nHistory of ...,Doctor: Can you tell me more about your family...
1235,952,952,PASTSURGICAL,Symptoms: N/A\r\nDiagnosis: N/A\r\nHistory of ...,Doctor: Did you have any surgeries in the past...


Only use train data to build the VDB (here we do not really train anything, but we do not leak test data into out rag or agent system)

In [ ]:
from df_as_search_db import get_any_case_df_as_vdb as get_db
from df_as_search_db import lazy_collate, collate_with_metadata, collate_with_metadata_with_output
from functools import partial

train_w_output_lazy = get_db(train_df,
    collate_func=partial(lazy_collate, cols=train_df.columns),
    # first_run=True,
    db_path="mts_train_w_output_lazy_vdb")

train_lazy = get_db(train_df,
    collate_func=partial(lazy_collate, cols=train_df.columns),
    # first_run=True,
    db_path="mts_train_lazy_vdb")


 - get ground truth data from test_df, we use claude to tidy them up.
 - here use langchain, and create a fallback and retry mechanism.

In [9]:
len(test_df)

64

In [10]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_core.callbacks import StdOutCallbackHandler
from langchain.callbacks.tracers import ConsoleCallbackHandler
from typing import List, Tuple
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from langchain import PromptTemplate, LLMChain
from langchain.chains.base import Chain

handler = ConsoleCallbackHandler()  # StdOutCallbackHandler()

In [11]:
def df_row_to_ground_truth_parse_control(question, llm):


    response_schemas = [
        ResponseSchema(
            name="Patient_known_knowledge",
            description="What patient knows about the symptoms and background information",
            type="string"
        ),
        ResponseSchema(
            name="Doctor_question_statements",
            description="All information doctor asked and stated.",
            type='string'
        ),
    ]

    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
    format_instructions = output_parser.get_format_instructions()

    template = """
    "
    {question}
    " 
    Above is a piece of collated patient note data, complete the task below
    {format_instructions}
    """

    prompt = PromptTemplate(
        input_variables=["question"],
        template=template,
        partial_variables={"format_instructions": format_instructions},
    )

    chain = prompt | llm | output_parser

    return chain.invoke(input={"question": question}, config={'callbacks': [handler]})

In [12]:
# df_row_to_ground_truth_parse_control(lazy_collate(train_df.iloc[0,:],  cols=train_df.columns  ), llm=claude)

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RunnableSequence > 2:prompt:PromptTemplate] Entering Prompt run with input:
[inputs]
[chain/end] [1:chain:RunnableSequence > 2:prompt:PromptTemplate] s] Exiting Prompt run with output:
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "prompts",
    "base",
    "StringPromptValue"
  ],
  "kwargs": {
    "text": "\n    \"\n    page_content='index: 137\\nID: 137\\nsection_header: PASTMEDICALHX\\nsection_text: Symptoms: N/A.\\r\\nDiagnosis: N/A.\\r\\nHistory of Patient: N/A.\\r\\nPlan of Action: N/A.\\r\\n\\ndialogue: Doctor: Have you ever been diagnosed with any illnesses? \\nPatient: No, I do not have any diseases.\\nDoctor: How about any issues with B P, thyroid, diabetes?\\nPatient: Nope none.' metadata={'row': index                                                           137\nID                                                              137\nsection_header   

{'Patient_known_knowledge': 'The patient reports no known illnesses, diseases, or issues with blood pressure, thyroid, or diabetes.',
 'Doctor_question_statements': 'The doctor asked if the patient has ever been diagnosed with any illnesses, and specifically inquired about issues with blood pressure, thyroid, or diabetes.'}

this is not bad, we can check hallucination by reflection but it is not discussed here.

In [15]:
ground_truth_test_df_claude = pd.read_csv('mts_ground_truth.csv', )
ground_truth_test_df_claude

,Patient_known_knowledge,Doctor_question_statements
0,The patient is a 5-year-old male with signific...,The doctor asked the age of the patient. Upon ...
1,"Patient is experiencing behavioral problems, d...",Doctor asked if the patient was referred by Do...
2,"The patient currently lives at home with mom, ...","The doctor asked who all are in the family, if..."
3,The patient has been experiencing tingling and...,The doctor asked the patient what brings them ...
4,The patient is a 54 year old female experienci...,"The doctor asked the patient's age, gender, sp..."
...,...,...
59,"Patient denies recent anxiety attacks, depress...",Doctor asked if the patient has had any anxiet...
60,Patient denies any issues or symptoms from hea...,Doctor asked if anything else was going on fro...
61,The patient is a 25-year-old female who has be...,"The doctor asked the patient's age, when she f..."
62,Symptoms: N/A. Diagnosis: N/A. History of Pati...,Doctor: Is he currently taking any medication?


Imitation of patient: based on all data provided by patient known knowledge, make up the first thing patient may say to the doctor in a clinic.
Note that one is not likely to dump all the information to the doctor at once. 


In [18]:
def ground_truth_to_first_question(patient_known_knowledge, llm):


    response_schemas = [
        ResponseSchema(
            name="Patient_first_statement",
            description="What patient knows about the symptoms and background information",
            type="string"
        ),
    ]

    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
    format_instructions = output_parser.get_format_instructions()

    template = """
    "
    {patient_known_knowledge}
    " 
    Above is all the infomation the patient knows. Using these data, make up a text statement to test a clinical application.
    Imitation of patient: based on all data provided by patient known knowledge, make up the first thing patient may say to the doctor in a clinic.
    Note that one is not likely to dump all the information to the doctor at once. 
    {format_instructions}
    """

    prompt = PromptTemplate(
        input_variables=["patient_known_knowledge"],
        template=template,
        partial_variables={"format_instructions": format_instructions},
    )

    chain = prompt | llm | output_parser

    return chain.invoke(input={"patient_known_knowledge": patient_known_knowledge}, config={'callbacks': [handler]})

In [19]:
ground_truth_to_first_question(ground_truth_test_df_claude.iloc[0,:]['Patient_known_knowledge'], local_llm)

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "patient_known_knowledge": "The patient is a 5-year-old male with significant feet pain and deformity in both feet. His feet appear to rotate inwards. The family is aware of the pain and the visible deformity."
}
[chain/start] [1:chain:RunnableSequence > 2:prompt:PromptTemplate] Entering Prompt run with input:
{
  "patient_known_knowledge": "The patient is a 5-year-old male with significant feet pain and deformity in both feet. His feet appear to rotate inwards. The family is aware of the pain and the visible deformity."
}
[chain/end] [1:chain:RunnableSequence > 2:prompt:PromptTemplate] s] Exiting Prompt run with output:
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "prompts",
    "base",
    "StringPromptValue"
  ],
  "kwargs": {
    "text": "\n    \"\n    The patient is a 5-year-old male with significant feet pain and deformity in both feet. His feet appear to rotate inwards. The family

{'Patient_first_statement': "Doctor, my feet really hurt and I've been having a hard time walking. I've noticed that my feet look different and seem to turn inwards."}

## Grading the relevance of question

 - relevance: how relevant is the new question to the patient's symptom? 
 - coverage: how much this new question cover the information of the full ground truth data?
 - fluency: is this question written in fluent, human language?
 - comments: other comments on the question from the system.



In [ ]:
def judge(patient_known_knowledge, patient_in, system_question, llm):


    response_schemas = [
        ResponseSchema(
            name="Relevance",
            description="How relevant is the new question to the patient's symptom? Rate from 1-5",
            type="string"
        ),

        ResponseSchema(
            name="Coverage",
            description="how much this new question cover the information of the full ground truth data? Rate from 1-5",
            type="string"
        ),

        ResponseSchema(
            name="Fluency",
            description="How fluent and human friendly is this question? Rate from 1-5",
            type="string"
        ),
        ResponseSchema(
            name="Comments",
            description=" other comments on the question from the system.",
            type="string"
        ),
    ]

    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
    format_instructions = output_parser.get_format_instructions()

    template = """
    "
    {patient_known_knowledge}
    " 
    Above is all the infomation the patient knows.
    
    The patient gives the application this information:
    {patient_in}

    The application comes up with this question:
    {system_question}

     Using these data, make up a text statement to test a clinical application.
     - relevance: how relevant is the new question to the patient's symptom? 
     - coverage: how much this new question cover the information of the full ground truth data?
     - fluency: is this question written in fluent, human language?    
     {format_instructions}
    """

    prompt = PromptTemplate(
        input_variables=["patient_known_knowledge"],
        template=template,
        partial_variables={"format_instructions": format_instructions},
    )

    chain = prompt | llm | output_parser

    return chain.invoke(input={"patient_known_knowledge": patient_known_knowledge}, config={'callbacks': [handler]})